In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
main_data = pd.read_csv('processed_imdb.csv')
main_data = main_data.drop(['Unnamed: 0'],axis=1)
e = []
for i in main_data['popularity']:
    try:
        e.append(float(i))
    except:
        e.append(0.0)

main_data['popularity'] = e
main_data.head()

,adult,id,original_language,popularity,revenue,runtime,status,video,vote_average,vote_count,genreset,prod_comp,prod_countries,status_int,lang_int
0,0,862,en,21.946943,373554033.0,81.0,Released,0,7.7,5415.0,"['Animation', 'Comedy', 'Family']",['Pixar Animation Studios'],['United States of America'],1,13
1,0,8844,en,17.015539,262797249.0,104.0,Released,0,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1,13
2,0,31357,en,3.859495,81452156.0,127.0,Released,0,6.1,34.0,"['Comedy', 'Drama', 'Romance']",['Twentieth Century Fox Film Corporation'],['United States of America'],1,13
3,0,11862,en,8.387519,76578911.0,106.0,Released,0,5.7,173.0,['Comedy'],"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],1,13
4,0,949,en,17.924927,187436818.0,170.0,Released,0,7.7,1886.0,"['Action', 'Crime', 'Drama', 'Thriller']","['Regency Enterprises', 'Forward Pass', 'Warne...",['United States of America'],1,13


In [5]:
r = main_data['revenue']
d = main_data.drop(['revenue'], axis=1)
main_data = pd.concat([d,r], axis=1)
main_data = main_data.dropna()
main_data.head()

,adult,id,original_language,popularity,runtime,status,video,vote_average,vote_count,genreset,prod_comp,prod_countries,status_int,lang_int,revenue
0,0,862,en,21.946943,81.0,Released,0,7.7,5415.0,"['Animation', 'Comedy', 'Family']",['Pixar Animation Studios'],['United States of America'],1,13,373554033.0
1,0,8844,en,17.015539,104.0,Released,0,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1,13,262797249.0
2,0,31357,en,3.859495,127.0,Released,0,6.1,34.0,"['Comedy', 'Drama', 'Romance']",['Twentieth Century Fox Film Corporation'],['United States of America'],1,13,81452156.0
3,0,11862,en,8.387519,106.0,Released,0,5.7,173.0,['Comedy'],"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],1,13,76578911.0
4,0,949,en,17.924927,170.0,Released,0,7.7,1886.0,"['Action', 'Crime', 'Drama', 'Thriller']","['Regency Enterprises', 'Forward Pass', 'Warne...",['United States of America'],1,13,187436818.0


In [6]:
main_data.dtypes

adult                  int64
id                    object
original_language     object
popularity           float64
runtime              float64
status                object
video                  int64
vote_average         float64
vote_count           float64
genreset              object
prod_comp             object
prod_countries        object
status_int             int64
lang_int               int64
revenue              float64
dtype: object

In [7]:
def normalize_data(col_name):
    X = main_data[col_name]
    X = (X-X.min())/(X.max()-X.min())
    return X

In [8]:
data = pd.DataFrame()

In [9]:
data['popularity'] = normalize_data('popularity')
data['vote_count'] = normalize_data('vote_average')
data['runtime'] = normalize_data('runtime')
data['revenue'] = normalize_data('revenue')
data.head()

,popularity,vote_count,runtime,revenue
0,0.040087,0.77,0.239645,0.133988
1,0.031079,0.69,0.307692,0.094261
2,0.007049,0.61,0.375740,0.029216
3,0.015320,0.57,0.313609,0.027468
4,0.032740,0.77,0.502959,0.067231


In [10]:
data.corr('pearson')

,popularity,vote_count,runtime,revenue
popularity,1.000000,0.16089,0.112463,0.457799
vote_count,0.160890,1.00000,0.296440,0.149010
runtime,0.112463,0.29644,1.000000,0.202650
revenue,0.457799,0.14901,0.202650,1.000000


In [11]:
train,test = train_test_split(data, test_size=0.25)

In [12]:
def input_func(df):
    label = df['revenue']
    ds = tf.data.Dataset.from_tensor_slices((dict(df),label))
    return ds

In [13]:
ds = input_func(train)
dt = input_func(test)

In [14]:
def input_train():
    return (ds.repeat().batch(5).make_one_shot_iterator().get_next())

def input_test():
    return (dt.repeat().batch(5).make_one_shot_iterator().get_next())

In [15]:
feature_columns = [
      tf.feature_column.numeric_column(key="popularity"),
      tf.feature_column.numeric_column(key="vote_count"),
      tf.feature_column.numeric_column(key="runtime")
]

In [16]:
model = tf.estimator.LinearRegressor(feature_columns=feature_columns, optimizer=tf.train.FtrlOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpiam7hp8f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000187CA2B7A90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
STEPS = 1000
model.train(input_fn=input_train, steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ADMINI~1\AppData\Local\Temp\tmpiam7hp8f\model.ckpt.
INFO:tensorflow:loss = 0.0020867009, step = 1
INFO:tensorflow:global_step/sec: 621.833
INFO:tensorflow:loss = 0.0058848173, step = 101 (0.173 sec)
INFO:tensorflow:global_step/sec: 1317.57
INFO:tensorflow:loss = 0.0039387834, step = 201 (0.064 sec)
INFO:tensorflow:global_step/sec: 1564.55
INFO:tensorflow:loss = 0.008651267, step = 301 (0.064 sec)
INFO:tensorflow:global_step/sec: 1564.61
INFO:tensorflow:loss = 0.014037794, step = 401 (0.068 sec)
INFO:tensorflow:global_step/sec: 1564.58
INFO:tensorflow:loss = 0.0047963904, step = 501 (0.064 sec)
INFO:tensorflow:global_step/sec: 1472.57
INFO:tensorflow:loss = 0.00027651584, step = 601 (0.064 sec)
INFO:

In [18]:
eval_ = model.evaluate(input_fn=input_test,steps=STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-21-12:05:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\tmpiam7hp8f\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2018-10-21-12:05:45
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.0027694104, global_step = 1000, label/mean = 0.02727934, loss = 0.013847051, prediction/mean = 0.02763483
INFO:tensorflow:Saving 'chec

In [19]:
# Mean Squared Error

mse = eval_['average_loss']
mse

0.0027694104

In [20]:
# Root Mean Squared Error 

np.sqrt(mse)

0.052625187

In [21]:
np.sqrt(mse)*main_data['revenue'].max()

146717183.3317449